# First Part (Excercises 1 - 4)

In [154]:
import pandas as pd

In [155]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [156]:
df = wiki[0]

In [157]:
df = df[df['Borough'] != 'Not assigned']

In [158]:
df.head()

Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights

In [159]:
df.shape

(211, 3)

# Second Part

In [160]:
cds = pd.read_csv('/Users/rogeliomj/Downloads/Geospatial_Coordinates.csv')

In [164]:
cds.rename(columns={'Postal Code':'Postcode','Neighbourhood':'Neighborhood'}, inplace=True)

In [162]:
df = pd.merge(df,cds, how='left', on='Postcode')

In [163]:
df.head()

Postcode           Borough     Neighbourhood   Latitude  Longitude
0      M3A        North York         Parkwoods  43.753259 -79.329656
1      M4A        North York  Victoria Village  43.725882 -79.315572
2      M5A  Downtown Toronto      Harbourfront  43.654260 -79.360636
3      M5A  Downtown Toronto       Regent Park  43.654260 -79.360636
4      M6A        North York  Lawrence Heights  43.718518 -79.464763

In [165]:
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

# Third Part: Clustering

In [166]:
df['finder'] = df['Borough'].str.find('Toronto')

In [167]:
df = df[df['finder'] != -1]

In [170]:
df.head()

Postcode           Borough     Neighborhood   Latitude  Longitude  finder
2       M5A  Downtown Toronto     Harbourfront  43.654260 -79.360636       9
3       M5A  Downtown Toronto      Regent Park  43.654260 -79.360636       9
13      M5B  Downtown Toronto          Ryerson  43.657162 -79.378937       9
14      M5B  Downtown Toronto  Garden District  43.657162 -79.378937       9
27      M5C  Downtown Toronto   St. James Town  43.651494 -79.375418       9

In [171]:
df.drop(columns='finder', inplace=True)

In [178]:
df = df.reset_index(drop=True)

## Map of Toronto with its boroughs

In [172]:
import numpy as np
import folium
import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [174]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.7001100, -79.4163000], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Import Foursquare Credentials

In [175]:
CLIENT_ID = 'X2BOT3ZFC3IHF0QVIQ4ZDRZC2BQBGYXOUI05KCTTS2QVUNDG' # your Foursquare ID
CLIENT_SECRET = '5IQ2I3LIBVSMU5FDBJZYZJWFVKVENIDHUGUBUD20W3H5FMXW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X2BOT3ZFC3IHF0QVIQ4ZDRZC2BQBGYXOUI05KCTTS2QVUNDG
CLIENT_SECRET:5IQ2I3LIBVSMU5FDBJZYZJWFVKVENIDHUGUBUD20W3H5FMXW


## Get nearby venues of Toronto's boroughs

In [180]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

# print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
#                                                               neighborhood_latitude, 
#                                                               neighborhood_longitude))

In [181]:
results = requests.get(url).json()

In [183]:

LIMIT = 60 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=X2BOT3ZFC3IHF0QVIQ4ZDRZC2BQBGYXOUI05KCTTS2QVUNDG&client_secret=5IQ2I3LIBVSMU5FDBJZYZJWFVKVENIDHUGUBUD20W3H5FMXW&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=60'

In [184]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d9ea232ad1789003965529d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 53,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [185]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [186]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories  \
0                                 Roselle Desserts                Bakery   
1                                    Tandem Coffee           Coffee Shop   
2  Toronto Cooper Koo Family Cherry St YMCA Centre  Gym / Fitness Center   
3                              Body Blitz Spa East                   Spa   
4                                   Impact Kitchen            Restaurant   

         lat        lng  
0  43.653447 -79.362017  
1  43.653559 -79.361809  
2  43.653191 -79.357947  
3  43.654735 -79.359874  
4  43.656369 -79.356980

In [187]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [189]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [190]:
print(toronto_venues.shape)
toronto_venues.head()

(2482, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Harbourfront               43.65426              -79.360636   
1  Harbourfront               43.65426              -79.360636   
2  Harbourfront               43.65426              -79.360636   
3  Harbourfront               43.65426              -79.360636   
4  Harbourfront               43.65426              -79.360636   

                                             Venue  Venue Latitude  \
0                                 Roselle Desserts       43.653447   
1                                    Tandem Coffee       43.653559   
2  Toronto Cooper Koo Family Cherry St YMCA Centre       43.653191   
3                              Body Blitz Spa East       43.654735   
4                                   Impact Kitchen       43.656369   

   Venue Longitude        Venue Category  
0       -79.362017                Bakery  
1       -79.361809           Coffee Shop  
2       -79.357947  Gym / Fitness Center  
3       -79.359874                   Spa  
4       -79.356980            Restaurant

In [191]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                               
Adelaide                                                              60   
Bathurst Quay                                                         15   
Berczy Park                                                           56   
Brockton                                                              23   
Business Reply Mail Processing Centre 969 Eastern                     18   
...                                                                  ...   
Underground city                                                      60   
Union Station                                                         60   
University of Toronto                                                 35   
Victoria Hotel                                                        60   
Yorkville                                                             21   

                                                   Neighborhood Longitude  \
Neighborhood                                                                
Adelaide                                                               60   
Bathurst Quay                                                          15   
Berczy Park                                                            56   
Brockton                                                               23   
Business Reply Mail Processing Centre 969 Eastern                      18   
...                                                                   ...   
Underground city                                                       60   
Union Station                                                          60   
University of Toronto                                                  35   
Victoria Hotel                                                         60   
Yorkville                                                              21   

                                                   Venue  Venue Latitude  \
Neighborhood                                                               
Adelaide                                              60              60   
Bathurst Quay                                         15              15   
Berczy Park                                           56              56   
Brockton                                              23              23   
Business Reply Mail Processing Centre 969 Eastern     18              18   
...                                                  ...             ...   
Underground city                                      60              60   
Union Station                                         60              60   
University of Toronto                                 35              35   
Victoria Hotel                                        60              60   
Yorkville                                             21              21   

                                                   Venue Longitude  \
Neighborhood                                                         
Adelaide                                                        60   
Bathurst Quay                                                   15   
Berczy Park                                                     56   
Brockton                                                        23   
Business Reply Mail Processing Centre 969 Eastern               18   
...                                                            ...   
Underground city                                                60   
Union Station                                                   60   
University of Toronto                                           35   
Victoria Hotel                                                  60   
Yorkville                                                       21   

                                                   Venue Category  
Neighborhood                                                       
Adelaide                

In [192]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 213 unique categories.


In [193]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0            0        0                   0             0               0   
1            0        0                   0             0               0   
2            0        0                   0             0               0   
3            0        0                   0             0               0   
4            0        0                   0             0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  ...  Theme Restaurant  Thrift / Vintage Store  Toy / Game Store  \
0         0  ...                 0                       0                 0   
1         0  ...                 0                       0                 0   
2         0  ...                 0                       0                 0   
3         0  ...                 0                       0                 0   
4         0  ...                 0                       0                 0   

   Trail  Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0      0              0                              0                 0   
1      0              0                              0                 0   
2      0              0                              0                 0   
3      0              0                              0                 0   
4      0              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  
0                      0         0            0  
1                      0         0            0  
2                      0         0            0  
3                      0         0            0  
4                      0         0            0  

[5 rows x 213 columns]

In [194]:
toronto_onehot.shape

(2482, 213)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [195]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio   Airport  \
0                                            Adelaide     0.000000  0.000000   
1                                       Bathurst Quay     0.000000  0.066667   
2                                         Berczy Park     0.000000  0.000000   
3                                            Brockton     0.043478  0.000000   
4   Business Reply Mail Processing Centre 969 Eastern     0.000000  0.000000   
..                                                ...          ...       ...   
68                                   Underground city     0.000000  0.000000   
69                                      Union Station     0.000000  0.000000   
70                              University of Toronto     0.000000  0.000000   
71                                     Victoria Hotel     0.000000  0.000000   
72                                          Yorkville     0.000000  0.000000   

    Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0             0.000000      0.000000        0.000000         0.000000   
1             0.066667      0.066667        0.133333         0.133333   
2             0.000000      0.000000        0.000000         0.000000   
3             0.000000      0.000000        0.000000         0.000000   
4             0.000000      0.000000        0.000000         0.000000   
..                 ...           ...             ...              ...   
68            0.000000      0.000000        0.000000         0.000000   
69            0.000000      0.000000        0.000000         0.000000   
70            0.000000      0.000000        0.000000         0.000000   
71            0.000000      0.000000        0.000000         0.000000   
72            0.000000      0.000000        0.000000         0.000000   

    Airport Terminal  American Restaurant  Antique Shop  ...  \
0           0.000000             0.066667           0.0  ...   
1           0.133333             0.000000           0.0  ...   
2           0.000000             0.000000           0.0  ...   
3           0.000000             0.000000           0.0  ...   
4           0.000000             0.000000           0.0  ...   
..               ...                  ...           ...  ...   
68          0.000000             0.033333           0.0  ...   
69          0.000000             0.000000           0.0  ...   
70          0.000000             0.000000           0.0  ...   
71          0.000000             0.033333           0.0  ...   
72          0.000000             0.047619           0.0  ...   

    Theme Restaurant  Thrift / Vintage Store  Toy / Game Store  Trail  \
0                0.0                     0.0               0.0    0.0   
1                0.0                     0.0               0.0    0.0   
2                0.0                     0.0               0.0    0.0   
3                0.0                     0.0               0.0    0.0   
4                0.0                     0.0               0.0    0.0   
..               ...                     ...               ...    ...   
68               0.0                     0.0               0.0    0.0   
69               0.0                     0.0               0.0    0.0   
70               0.0                     0.0               0.0    0.0   
71               0.0                     0.0               0.0    0.0   
72               0.0                     0.0               0.0    0.0   

    Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0        0.000000                       0.016667          0.000000   
1        0.000000                       0.000000          0.000000   
2        0.000000                       0.017857          0.000000   
3        0.000000                       0.000000          0.000000   
4        0.000000                       0.000000          0.000000   
..            ...                            ...               ...   
68       0.000000                       0.016667          0.000000 

In [196]:
toronto_grouped.shape

(73, 213)

In [197]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.08
1           Steakhouse  0.07
2  American Restaurant  0.07
3     Asian Restaurant  0.05
4                 Café  0.05


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3          Boutique  0.07
4           Airport  0.07


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Steakhouse  0.04
3          Café  0.04
4   Cheese Shop  0.04


----Brockton----
                    venue  freq
0          Breakfast Spot  0.09
1                    Café  0.09
2             Coffee Shop  0.09
3             Yoga Studio  0.04
4  Furniture / Home Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1           Pizza Place  0.06
2            Restaurant  0.06
3  Fast Food Restaurant  0.06
4            Smoke Shop  0.06


----CN Tower----
     



----South Hill----
                venue  freq
0         Coffee Shop  0.12
1                 Pub  0.12
2  Light Rail Station  0.06
3  Athletics & Sports  0.06
4          Restaurant  0.06


----South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3          Boutique  0.07
4           Airport  0.07


----St. James Town----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.06
2          Restaurant  0.05
3  Italian Restaurant  0.05
4           Gastropub  0.04


----Stn A PO Boxes 25 The Esplanade----
            venue  freq
0            Café  0.07
1      Restaurant  0.07
2    Cocktail Bar  0.05
3     Coffee Shop  0.05
4  Farmers Market  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----Summerhill East----
                      venu

#### Let's put that into a *pandas* dataframe

In [198]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [199]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                           Adelaide           Coffee Shop   
1                                      Bathurst Quay        Airport Lounge   
2                                        Berczy Park           Coffee Shop   
3                                           Brockton           Coffee Shop   
4  Business Reply Mail Processing Centre 969 Eastern    Light Rail Station   

  2nd Most Common Venue 3rd Most Common Venue   4th Most Common Venue  \
0            Steakhouse   American Restaurant                    Café   
1       Airport Service      Airport Terminal         Harbor / Marina   
2          Cocktail Bar        Farmers Market                  Bakery   
3        Breakfast Spot                  Café  Furniture / Home Store   
4               Brewery           Pizza Place          Farmers Market   

  5th Most Common Venue 6th Most Common Venue  7th Most Common Venue  \
0      Asian Restaurant                   Bar                  Hotel   
1               Airport    Airport Food Court           Airport Gate   
2              Beer Bar    Seafood Restaurant            Cheese Shop   
3            Restaurant             Pet Store  Performing Arts Venue   
4               Butcher            Restaurant          Auto Workshop   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0             Gastropub      Sushi Restaurant            Pizza Place  
1              Boutique                   Bar          Boat or Ferry  
2                  Café            Steakhouse                   Park  
3    Italian Restaurant          Intersection                    Gym  
4                   Spa            Smoke Shop   Fast Food Restaurant

## Cluster Neighborhoods

In [200]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

In [201]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postcode           Borough     Neighborhood   Latitude  Longitude  \
0      M5A  Downtown Toronto     Harbourfront  43.654260 -79.360636   
1      M5A  Downtown Toronto      Regent Park  43.654260 -79.360636   
2      M5B  Downtown Toronto          Ryerson  43.657162 -79.378937   
3      M5B  Downtown Toronto  Garden District  43.657162 -79.378937   
4      M5C  Downtown Toronto   St. James Town  43.651494 -79.375418   

   Cluster Labels 1st Most Common Venue      2nd Most Common Venue  \
0               0           Coffee Shop                       Café   
1               0           Coffee Shop                       Café   
2               0           Coffee Shop  Middle Eastern Restaurant   
3               0           Coffee Shop  Middle Eastern Restaurant   
4               0           Coffee Shop                       Café   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                Bakery                  Park    Mexican Restaurant   
1                Bakery                  Park    Mexican Restaurant   
2                  Café  Fast Food Restaurant                Bakery   
3                  Café  Fast Food Restaurant                Bakery   
4            Restaurant    Italian Restaurant             Gastropub   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0        Breakfast Spot            Restaurant                   Pub   
1        Breakfast Spot            Restaurant                   Pub   
2               Theater            Restaurant        Cosmetics Shop   
3               Theater            Restaurant        Cosmetics Shop   
4           Pizza Place   Japanese Restaurant                Bakery   

  9th Most Common Venue 10th Most Common Venue  
0               Theater            Yoga Studio  
1               Theater            Yoga Studio  
2        Clothing Store              Bookstore  
3        Clothing Store              Bookstore  
4                  Park              BBQ Joint

#### Finally, let's visualize the resulting clusters

In [205]:
# create map
map_clusters = folium.Map(location=[43.7001100, -79.4163000], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

### Cluster 0

In [214]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0   Downtown Toronto               0           Coffee Shop   
1   Downtown Toronto               0           Coffee Shop   
2   Downtown Toronto               0           Coffee Shop   
3   Downtown Toronto               0           Coffee Shop   
4   Downtown Toronto               0           Coffee Shop   
5       East Toronto               0           Coffee Shop   
6   Downtown Toronto               0           Coffee Shop   
7   Downtown Toronto               0           Coffee Shop   
8   Downtown Toronto               0         Grocery Store   
9   Downtown Toronto               0           Coffee Shop   
10  Downtown Toronto               0           Coffee Shop   
11  Downtown Toronto               0           Coffee Shop   
12      West Toronto               0              Pharmacy   
13      West Toronto               0              Pharmacy   
14  Downtown Toronto               0           Coffee Shop   
15  Downtown Toronto               0           Coffee Shop   
16  Downtown Toronto               0           Coffee Shop   
17      West Toronto               0                   Bar   
18      West Toronto               0                   Bar   
19      East Toronto               0      Greek Restaurant   
20      East Toronto               0      Greek Restaurant   
21  Downtown Toronto               0           Coffee Shop   
22  Downtown Toronto               0           Coffee Shop   
23      West Toronto               0           Coffee Shop   
24      West Toronto               0           Coffee Shop   
25      West Toronto               0           Coffee Shop   
28  Downtown Toronto               0           Coffee Shop   
29  Downtown Toronto               0           Coffee Shop   
30      East Toronto               0                  Café   
36      West Toronto               0                  Café   
37      West Toronto               0                  Café   
38   Central Toronto               0        Clothing Store   
39   Central Toronto               0                  Café   
40   Central Toronto               0                  Café   
41   Central Toronto               0                  Café   
42      West Toronto               0             Gift Shop   
43      West Toronto               0             Gift Shop   
44   Central Toronto               0        Sandwich Place   
45  Downtown Toronto               0                  Café   
46  Downtown Toronto               0                  Café   
47      West Toronto               0                  Café   
48      West Toronto               0                  Café   
51  Downtown Toronto               0                  Café   
52  Downtown Toronto               0                  Café   
53  Downtown Toronto               0                  Café   
54   Central Toronto               0                   Pub   
55   Central Toronto               0                   Pub   
56   Central Toronto               0                   Pub   
57   Central Toronto               0                   Pub   
58   Central Toronto               0                   Pub   
67  Downtown Toronto               0                  Café   
68  Downtown Toronto               0           Coffee Shop   
69  Downtown Toronto               0           Coffee Shop   
70  Downtown Toronto               0           Coffee Shop   
71  Downtown Toronto               0           Coffee Shop   
72  Downtown Toronto               0           Coffee Shop   
73      East Toronto               0    Light Rail Station   

            2nd Most Common Venue 3rd Most Common Venue  \
0                            Café                Bakery   
1                            Café                Bakery   
2       Middle Eastern Restaurant                  Café   
3       Middle Eastern Restaurant                  Café   
4                            Café            Restaurant   
5               Health Food Store                 Trail   
6  

### Cluster 1

In [209]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
26      East Toronto               1                  Park   
27      East Toronto               1                  Park   
31   Central Toronto               1              Bus Line   
33   Central Toronto               1        Breakfast Spot   
34   Central Toronto               1              Bus Line   
35   Central Toronto               1              Bus Line   
66  Downtown Toronto               1                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
26        Ice Cream Shop                   Pub        Sandwich Place   
27        Ice Cream Shop                   Pub        Sandwich Place   
31                  Park           Swim School      Cuban Restaurant   
33                 Hotel                  Park                   Gym   
34                  Park                 Trail         Jewelry Store   
35                  Park                 Trail         Jewelry Store   
66            Playground                 Trail              Building   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
26               Brewery          Burger Joint         Burrito Place   
27               Brewery          Burger Joint         Burrito Place   
31   Dumpling Restaurant            Donut Shop               Dog Run   
33        Clothing Store     Food & Drink Shop        Sandwich Place   
34      Sushi Restaurant          Cupcake Shop   Dumpling Restaurant   
35      Sushi Restaurant          Cupcake Shop   Dumpling Restaurant   
66           Wings Joint      Cuban Restaurant            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
26    Italian Restaurant          Intersection   Fast Food Restaurant  
27    Italian Restaurant          Intersection   Fast Food Restaurant  
31        Discount Store                 Diner           Dessert Shop  
33     Convenience Store      Department Store           Dessert Shop  
34            Donut Shop               Dog Run         Discount Store  
35            Donut Shop               Dog Run         Discount Store  
66               Dog Run        Discount Store                  Diner

### Cluster 2

In [210]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
49  Central Toronto               2            Playground   
50  Central Toronto               2            Playground   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
49     Electronics Store   Dumpling Restaurant            Donut Shop   
50     Electronics Store   Dumpling Restaurant            Donut Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
49               Dog Run        Discount Store                 Diner   
50               Dog Run        Discount Store                 Diner   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
49          Dessert Shop      Department Store          Deli / Bodega  
50          Dessert Shop      Department Store          Deli / Bodega

### Cluster 3

In [216]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
59  Downtown Toronto               3        Airport Lounge   
60  Downtown Toronto               3        Airport Lounge   
61  Downtown Toronto               3        Airport Lounge   
62  Downtown Toronto               3        Airport Lounge   
63  Downtown Toronto               3        Airport Lounge   
64  Downtown Toronto               3        Airport Lounge   
65  Downtown Toronto               3        Airport Lounge   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
59       Airport Service      Airport Terminal       Harbor / Marina   
60       Airport Service      Airport Terminal       Harbor / Marina   
61       Airport Service      Airport Terminal       Harbor / Marina   
62       Airport Service      Airport Terminal       Harbor / Marina   
63       Airport Service      Airport Terminal       Harbor / Marina   
64       Airport Service      Airport Terminal       Harbor / Marina   
65       Airport Service      Airport Terminal       Harbor / Marina   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
59               Airport    Airport Food Court          Airport Gate   
60               Airport    Airport Food Court          Airport Gate   
61               Airport    Airport Food Court          Airport Gate   
62               Airport    Airport Food Court          Airport Gate   
63               Airport    Airport Food Court          Airport Gate   
64               Airport    Airport Food Court          Airport Gate   
65               Airport    Airport Food Court          Airport Gate   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
59              Boutique                   Bar          Boat or Ferry  
60              Boutique                   Bar          Boat or Ferry  
61              Boutique                   Bar          Boat or Ferry  
62              Boutique                   Bar          Boat or Ferry  
63              Boutique                   Bar          Boat or Ferry  
64              Boutique                   Bar          Boat or Ferry  
65              Boutique                   Bar          Boat or Ferry

### Cluster 4

In [212]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
32  Central Toronto               4                Garden   

   2nd Most Common Venue 3rd Most Common Venue        4th Most Common Venue  \
32           Wings Joint      Cuban Restaurant  Eastern European Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
32   Dumpling Restaurant            Donut Shop               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
32        Discount Store                 Diner           Dessert Shop

### Cluster 5

In [213]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Empty DataFrame
Columns: [Borough, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []